In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

import sys
sys.path.append('../')
from config import Config, Shape
from plate import get_base, get_screw_positions, get_key_positions, make_plate
from case import get_basic_shape,  cut_aviator_connector_hole, get_distance_between_two_vertices,  add_microcontrollerbox

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [2]:
def make_controller_box_top_plate(case,config):

    controllerBoxLength =  get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<X'))['y']
    controllerBoxWidth = get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<Y'))['x']

    controller_box_top = (case
                        .faces(tag='controllerBoxTop')
                        .workplane(centerOption="CenterOfBoundBox", invert=True)
                        .move(0,config.wallThickness)
                        .box(controllerBoxWidth,controllerBoxLength, config.controllerBoxThickness, centered=[True,True,False],combine=False)
    )


    return controller_box_top

In [2]:

rkn = [4,8,8,6]
config=Config(rkn, shape=Shape.LEAN, floorThickness=3)
kp = get_key_positions(config)

In [10]:
case= get_basic_shape(config)

case = case.edges('|Z').fillet(3)

case.edges(tag='topEdge').val().Center().y

71.8

In [34]:

from cadquery import Selector
class CustomisableSelector(Selector):
    def set_filter(self, filter_function):
        self.filter = filter_function



In [93]:
ff = CustomisableSelector()

def filter_func(objectList):
        r = []
        for o in objectList:
            if o.Center().y > case.edges(tag='topEdge').val().Center().y  :
                r.append(o)
        return r

ff.set_filter(filter_func)

a=case.edges('>Z').edges(ff).wire().toPending()#.extrude(1)

In [186]:
edges = case.faces('>Z').edges(ff).toPending()#.vals()
edges
#case.edges('>Z').edges(ff).wires().val().assembleEdges(edges)

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.02s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [95]:
a.vertices()

[v.toTuple()[0] for v in a.vertices().vals()]

[35.5, 38.5, 38.5, 41.5, 97.5, 94.5, 94.5, 91.5]

In [182]:
ss = cq.Sketch()

for e in edges.vals(): ss.edge(e)


#ss.close()
xs = [v.toTuple()[0] for v in a.vertices().vals()]
ys = [v.toTuple()[1] for v in a.vertices().vals()]

ss
#print(min(ys), max(xs), min(xs))

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [119]:
max(xs), min(ys)

(97.5, 71.8)

In [184]:
#wp=case.faces('>Z').workplane(centerOption="CenterOfBoundBox")#cq.Workplane('XZ')
wp=cq.Workplane('XY')
#edges = sorted(a.vals(), key = lambda e: e.Center().y, reverse=True)
for e in edges.vals():
    wp.add(e)#.placeSketch(ss)
#wp.circle(10)

#wp.moveTo(0, 0)
#wp.line(0,20)
#wp.close()


print([v.toTuple() for v in wp.vertices().vals()])
#wp.moveTo(35.5, 71.8)
wp.line(0,10)

[(35.5, 71.8, 22.0), (38.5, 74.8, 22.0), (38.5, 86.8, 22.0), (41.5, 89.8, 22.0), (97.5, 71.8, 22.0), (94.5, 74.8, 22.0), (94.5, 86.8, 22.0), (91.5, 89.8, 22.0)]
100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [148]:
for v in a.vertices().vals():
    y_val = (v.toTuple()[1])
    if y_val == min(ys):
        print('found')
        v.tag('xxx')
        break

found


AttributeError: 'Vertex' object has no attribute 'tag'

In [7]:


if True:
    controllerBoxLength =  get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<X'))['y']
    controllerBoxWidth = get_distance_between_two_vertices(case.faces(tag='controllerBoxTop').vertices('<Y'))['x']+10

    controller_box_top = (case
                        .faces(tag='controllerBoxTop')
                        .workplane(centerOption="CenterOfBoundBox", invert=True)
                        .move(0,config.wallThickness)
                        .box(controllerBoxWidth,controllerBoxLength, config.controllerBoxThickness, centered=[True,True,False],combine=False)
    )

#case.intersect(controller_box_top)
    


In [9]:
case.faces('+Y',tag='outface').faces('>>Y[-2]').tag('xxxx')


case.faces(tag='xxxx').workplane(centerOption="CenterOfBoundBox").box(60,20,25, centered=[True,True,False])

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [3]:


def make_case(config:Config) -> cq.Sketch:
    """TODO: add screw holes for microcontroller (check they are in right position!!)
            change fillet parameters
            add module for leds, encoder and aviator
            """
    
    
    case= get_basic_shape(config)
    case.faces('<Z or >Z').tag('bottomface')


    # add box for microcontroller
    if config.controller is not None:
        case = add_microcontrollerbox(case, config)


    #fillet edges
    case = case.edges('|Z').fillet(config.edgeFillet)
    #case.edges('>Z').tag('outerTopEdge')
    case.faces('(not <Z) and (not >Z)').tag('outface')
    case.wires('>Z').all()[0].tag('topwire')

    # scoop out interior
    case= (case.faces(">Z or <Z").shell(-config.wallThickness, kind='intersection'))
    
    # set floor thickness 
    case = case.faces('<Z').wires('>X and >Y').toPending().extrude(config.floorThickness)
    
    return case
    
    #fillet bottom edge
    case = case.edges('<Z').fillet(3)



    # add plate screw holes 
    scr_hls = get_screw_positions(config)
    case=(case.faces('<Z').workplane(origin=(0,0))
          .pushPoints([(x,-y) for x,y in scr_hls])
          .cskHole(2.4, 4.8, 82, depth=None)    
          )

    # add controllerbox holes.
    # TODO - remove hardcoding on screw hole size
    if config.controller is not None:
        case = (case.workplaneFromTagged("controllerBox")
                .rect(config.controller.screw_hole_x,
                        config.controller.screw_hole_y- config.wallThickness - config.controllerYOffset - (config.controller.board_dimension_y -config.controller.screw_hole_y)*0.5, 
                        forConstruction=True ,centered=[True,True,True])
                .vertices()
                .cskHole(2.4, 4.8, 82, depth=None))
    
    #cut hole for aviator connector 
    case = cut_aviator_connector_hole(case, config)

    return case


In [4]:
case = make_case(config)

In [273]:
case.faces(cq.TypeSelector('CYLINDER'))#.faces('+Y')

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.03s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [158]:
case.faces('-Y').faces('>>Y').edges('>Z').tag('topPoints')

top_edge_of_box = case.edges(tag='topPoints').val()
vertices_of_box = [v.toTuple() for v in top_edge_of_box.Vertices()]

# find wire with edge in 

wires_to_search = case.wires('>Z').vals()


wire = list(filter(lambda x: top_edge_of_box in x.Edges(), wires_to_search))[0]


#filter wires connected to top edge of box

wanted_edges=[]
for e in wire.Edges():
    start_end_points = [v.toTuple() for v in e.Vertices()]
    if vertices_of_box[0] in start_end_points or  vertices_of_box[1] in start_end_points:
        wanted_edges.append(e)

    

In [188]:
edges_to_search =wire.Edges() 
starting_edge = top_edge_of_box

wanted_edges=[]

if True:
#while len(wanted_edges)<3:
    for edge in edges_to_search:
        start_end_points = [v.toTuple() for v in edge.Vertices()]
        starting_edge_vertex_start = starting_edge.Vertices()[0].toTuple()
        starting_edge_vertex_end = starting_edge.Vertices()[1].toTuple()

        if starting_edge_vertex_start in start_end_points or  starting_edge_vertex_end in start_end_points:
            wanted_edges.append(edge)
            starting_edge = edge
            if len(wanted_edges)>=4:
                break

In [228]:
start_end_points

[(41.5, 88.19999999999999, 22.0), (40.1, 86.8, 22.0)]

In [245]:
edges_to_search =wire.Edges() 
starting_edge = top_edge_of_box




wanted_edges=[starting_edge ]
idx=0
while len(wanted_edges)<=3:
    for edge in edges_to_search:
        start_end_points = [v.toTuple() for v in edge.Vertices()]
        starting_edge_vertex_start = starting_edge.Vertices()[0].toTuple()
        starting_edge_vertex_end = starting_edge.Vertices()[1].toTuple()
        #starting_edge_vertexes = [v.toTuple() for v in starting_edge.Vertices()]
        #edge_to_match = starting_edge_vertexes[idx]
        if starting_edge_vertex_start in start_end_points or starting_edge_vertex_end in start_end_points:
            wanted_edges.append(edge)
            starting_edge = edge
            edges_to_search.remove(edge)
            #edges_to_search.reverse()
            break
            


In [253]:
case.newObject(wanted_edges).wire()

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [249]:
wanted_edges

In [250]:
cq.Wire().combine(wanted_edges)

TypeError: Shape.__init__() missing 1 required positional argument: 'obj'

In [263]:
cq.Workplane('XY').newObject(wanted_edges).mirror('YZ')

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…